In [ ]:
pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 6.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import emoji

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Embedding



In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
data = pd.read_csv('/content/emoji_data.csv',header =None)
data.head()

,0,1
0,French macaroon is so tasty,4
1,work is horrible,3
2,I am upset,3
3,throw the ball,1
4,Good joke,2


In [ ]:
emoji_dict = {
    0: ":red_heart:",
    1: ":baseball:",
    2: ":grinning_face_with_big_eyes:",
    3: ":disappointed_face:",
    4: ":fork_and_knife_with_plate:"
}

def label_to_emoji(label):
  return emoji.emojize(emoji_dict[label])


In [ ]:
X= data[0].values
Y= data[1].values
Y[29]='1'
print(Y)

['4' '3' '3 ' '1 ' '2' '1' '4' '3' '4' '1' '3' '3 ' '2' '2' '4' '3' '2'
 '3 ' '3 ' '1' '3 ' '2' '2' '2' '0' '1' '0' '4 ' '2' '1' '2' '0' '0' '3 '
 '4' '0' '2' '1' '3' '1' '0' '4' '0 ' '3' '0 ' '4' '2' '3 ' '4' '2 ' '2'
 '3' '0' '2' '2' '3 ' '2' '3' '2' '2' '3 ' '3' '0 ' '2' '3' '0' '2' '0'
 '0 ' '2' '3' '2' '4' '1' '3' '3' '0' '0' '3' '2' '0' '3' '0' '2' '2' '4'
 '2' '2' '0' '0' '2' '3' '0' '4' '2' '1' '2' '3' '3' '2' '3' '0' '3' '0'
 '2' '0' '2' '3' '4' '3' '1' '3' '4' '3' '2' '3' '3' '3' '1' '4' '4' '2'
 '2' '1' '1' '2' '3' '2' '3' '4' '2' '3' '0' '2' '0' '0' '4' '3' '4' '2'
 '3' '2' '3' '4' '2' '1' '2' '4' '3' '1' '3' '2' '3' '2' '2' '3' '3' '2'
 '4' '0' '0' '0' '3' '0' '0' '1' '1' '2' '2' '2' '0' '3' '2' '3' '3' '1'
 '2' '2' '4' '2' '3' '1' '2']


Embeddings

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2023-10-09 04:50:02--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-10-09 04:50:02--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-10-09 04:50:02--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
file = open('glove.6B.100d.txt', 'r', encoding = 'utf8')
content = file.readlines()
file.close()


In [ ]:
embeddings = {}

for line in content:
    line = line.split()
    embeddings[line[0]] = np.array(line[1:], dtype = float)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
word2index = tokenizer.word_index

In [ ]:
Xtokens = tokenizer.texts_to_sequences(X)

def get_maxlen(data):
    maxlen = 0
    for sent in data:
        maxlen = max(maxlen, len(sent))
    return maxlen

maxlen = get_maxlen(Xtokens)
print(maxlen)
Xtrain = pad_sequences(Xtokens, maxlen = maxlen,  padding = 'post', truncating = 'post')



10


In [ ]:
Ytrain = to_categorical(Y)

Model

In [ ]:
embed_size = 100
embedding_matrix = np.zeros((len(word2index)+1, embed_size))

for word, i in word2index.items():
    embed_vector = embeddings[word]
    embedding_matrix[i] = embed_vector

In [ ]:
embedding_matrix

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [-0.046539,  0.61966 ,  0.56647 , ..., -0.37616 , -0.032502,
         0.8062  ],
       [-0.49886 ,  0.76602 ,  0.89751 , ..., -0.41179 ,  0.40539 ,
         0.78504 ],
       ...,
       [-0.46263 ,  0.069864,  0.69095 , ..., -0.29174 ,  0.32041 ,
         0.21202 ],
       [ 0.073242,  0.11134 ,  0.62281 , ...,  0.53417 , -0.1646  ,
        -0.27516 ],
       [ 0.29019 ,  0.80497 ,  0.31187 , ..., -0.33603 ,  0.45998 ,
        -0.11278 ]])

In [ ]:
model = Sequential([
    Embedding(input_dim = len(word2index) + 1,
              output_dim = embed_size,
              input_length = maxlen,
              weights = [embedding_matrix],
              trainable = False
             ),

    LSTM(units = 16, return_sequences = True),
    LSTM(units = 4),
    Dense(5, activation = 'softmax')
])

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(Xtrain, Ytrain, epochs = 100)

Epoch 1/100
6/6 [==============================] - 6s 14ms/step - loss: 1.6016 - accuracy: 0.2240
Epoch 2/100
6/6 [==============================] - 0s 12ms/step - loss: 1.5785 - accuracy: 0.2896
Epoch 3/100
6/6 [==============================] - 0s 13ms/step - loss: 1.5576 - accuracy: 0.3279
Epoch 4/100
6/6 [==============================] - 0s 13ms/step - loss: 1.5441 - accuracy: 0.3497
Epoch 5/100
6/6 [==============================] - 0s 15ms/step - loss: 1.5285 - accuracy: 0.3388
Epoch 6/100
6/6 [==============================] - 0s 15ms/step - loss: 1.5141 - accuracy: 0.3607
Epoch 7/100
6/6 [==============================] - 0s 14ms/step - loss: 1.4985 - accuracy: 0.3770
Epoch 8/100
6/6 [==============================] - 0s 13ms/step - loss: 1.4781 - accuracy: 0.3716
Epoch 9/100
6/6 [==============================] - 0s 13ms/step - loss: 1.4545 - accuracy: 0.3825
Epoch 10/100
6/6 [==============================] - 0s 14ms/step - loss: 1.4275 - accuracy: 0.3770
Epoch 11/100
6/6 [=

In [ ]:
test = ["I feel good", "I feel very bad", "lets eat dinner"]

test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq, maxlen = maxlen, padding = 'post', truncating = 'post')

y_pred = model.predict(Xtest)
y_pred = np.argmax(y_pred, axis = 1)

for i in range(len(test)):
    print(test[i], label_to_emoji(y_pred[i]))

1/1 [==============================] - 1s 910ms/step
I feel good 😃
I feel very bad 😞
lets eat dinner 🍽️
